In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}



!mkdir -p drive

!google-drive-ocamlfuse drive

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131331 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.6-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [2]:
!unzip -q drive/images.zip
!ls

adc.json  drive  images  sample_data  test.txt	train.txt  val.txt


In [0]:
import tensorflow as tf
from tensorflow.python.training.moving_averages import assign_moving_average
import numpy as np
import os, pdb
import cv2
import numpy as np
import random as rn
import tensorflow as tf
import threading
import time

global n_classes, ema_gp
ema_gp = []
n_classes = 50

In [0]:
def activation(x,name="activation"):
    return tf.nn.relu(x, name=name)
    
def conv2d(name, l_input, w, b, s, p):
    l_input = tf.nn.conv2d(l_input, w, strides=[1,s,s,1], padding=p, name=name)
    l_input = l_input+b

    return l_input

def batchnorm(conv, isTraining, name='bn'):
    return tf.layers.batch_normalization(conv, training=isTraining, name="bn"+name)

def initializer(in_filters, out_filters, name):
    w1 = tf.get_variable(name+"W", [3, 3, in_filters, out_filters], initializer=tf.truncated_normal_initializer())
    b1 = tf.get_variable(name+"B", [out_filters], initializer=tf.truncated_normal_initializer())
    return w1, b1
  
def residual_block(in_x, in_filters, out_filters, stride, isDownSampled, name, isTraining):
    global ema_gp
    # first convolution layer
    if isDownSampled:
      in_x = tf.nn.avg_pool(in_x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
      
    x = batchnorm(in_x, isTraining, name=name+'FirstBn')
    x = activation(x)
    w1, b1 = initializer(in_filters, in_filters, name+"first_res")
    x = conv2d(name+'r1', x, w1, b1, 1, "SAME")

    # second convolution layer
    x = batchnorm(x, isTraining, name=name+'SecondBn')
    x = activation(x)
    w2, b2 = initializer(in_filters, out_filters, name+"Second_res")
    x = conv2d(name+'r2', x, w2, b2, 1, "SAME")
    
    if in_filters != out_filters:
        difference = out_filters - in_filters
        left_pad = difference // 2
        right_pad = difference - left_pad
        identity = tf.pad(in_x, [[0, 0], [0, 0], [0, 0], [left_pad, right_pad]])
        return x + identity
    else:
        return in_x + x

      
def ResNet(_X, isTraining):
    global n_classes
    w1 = tf.get_variable("initW", [7, 7, 3, 96], initializer=tf.truncated_normal_initializer())
    b1 = tf.get_variable("initB", [96], initializer=tf.truncated_normal_initializer())
    x = conv2d('conv1', _X, w1, b1, 4, "VALID")
    
    filters_num = [96,128,256,384]
    block_num = [2,2,2,2]
    l_cnt = 1
    for i in range(len(filters_num)):
      for j in range(block_num[i]):
          
          if ((j==block_num[i]-1) & (i<len(filters_num)-1)):
            x = residual_block(x, filters_num[i], filters_num[i+1], 2, True, 'ResidualBlock%d'%(l_cnt), isTraining)
            print('[L-%d] Build %dth connection layer %d from %d to %d channels' % (l_cnt, i, j, filters_num[i], filters_num[i+1]))
          else:
            x = residual_block(x, filters_num[i], filters_num[i], 1, False, 'ResidualBlock%d'%(l_cnt), isTraining)
            print('[L-%d] Build %dth residual block %d with %d channels' % (l_cnt,i, j, filters_num[i]))
          l_cnt +=1

    
    wo, bo=initializer(filters_num[-1], n_classes, "FinalOutput")
    x = conv2d('final', x, wo, bo, 1, "SAME")
    
    x = activation(x)
    x = batchnorm(x, isTraining, name='FinalBn')
    
    x = tf.reduce_mean(x, [1,2])
    W = tf.get_variable("FinalW", [n_classes, n_classes], initializer=tf.truncated_normal_initializer())
    b = tf.get_variable("FinalB", [n_classes], initializer=tf.truncated_normal_initializer())
    
    out = tf.matmul(x, W) + b
                            

    return out

In [0]:
#==========================================================================
#=============Reading data in multithreading manner========================
#==========================================================================
def read_labeled_image_list(image_list_file, training_img_dir):
    """Reads a .txt file containing pathes and labeles
    Args:
       image_list_file: a .txt file with one /path/to/image per line
       label: optionally, if set label will be pasted after each line
    Returns:
       List with all filenames in file image_list_file
    """
    f = open(image_list_file, 'r')
    filenames = []
    labels = []

    for line in f:
        filename, label = line[:-1].split(' ')
        filename = training_img_dir+filename
        filenames.append(filename)
        labels.append(int(label))
        
    return filenames, labels
    
    
def read_images_from_disk(input_queue, size1=256):
    """Consumes a single filename and label as a ' '-delimited string.
    Args:
      filename_and_label_tensor: A scalar string tensor.
    Returns:
      Two tensors: the decoded image, and the string label.
    """
    label = input_queue[1]
    fn=input_queue[0]
    file_contents = tf.read_file(input_queue[0])
    example = tf.image.decode_jpeg(file_contents, channels=3)
    
    #example = tf.image.decode_png(file_contents, channels=3, name="dataset_image") # png fo rlfw
    example=tf.image.resize_images(example, [size1,size1])
    return example, label, fn
    
def setup_inputs(sess, filenames, training_img_dir, image_size=256, crop_size=224, isTest=False, batch_size=128):
    
    # Read each image file
    image_list, label_list = read_labeled_image_list(filenames, training_img_dir)

    images = tf.cast(image_list, tf.string)
    labels = tf.cast(label_list, tf.int64)
     # Makes an input queue
    if isTest is False:
        isShuffle = True
        numThr = 4
    else:
        isShuffle = False
        numThr = 1
        
    input_queue = tf.train.slice_input_producer([images, labels], shuffle=isShuffle)
    image, y,fn = read_images_from_disk(input_queue)

    channels = 3
    image.set_shape([None, None, channels])
        
    # Crop and other random augmentations
    if isTest is False:
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_saturation(image, .95, 1.05)
        image = tf.image.random_brightness(image, .05)
        image = tf.image.random_contrast(image, .95, 1.05)
        

    image = tf.random_crop(image, [crop_size, crop_size, 3])
    image = tf.cast(image, tf.float32)/255.0
    
    image, y,fn = tf.train.batch([image, y, fn], batch_size=batch_size, capacity=batch_size*3, num_threads=numThr, name='labels_and_images')

    tf.train.start_queue_runners(sess=sess)

    return image, y, fn, len(label_list)

In [6]:
batch_size = 256
display_step = 80
learning_rate = tf.placeholder(tf.float32)      # Learning rate to be fed
lr = 1e-2                          # Learning rate start
print('GO!!')

GO!!


In [7]:
# Setup the tensorflow...
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

print("Preparing the training & validation data...")
train_data, train_labels, filelist1, glen1 = setup_inputs(sess, "train.txt", "./", batch_size=batch_size)
val_data, val_labels, filelist2, tlen1 = setup_inputs(sess, "val.txt", "./", batch_size=batch_size,isTest=True)


max_iter = glen1*100
print("Preparing the training model with learning rate = %.5f..." % (lr))


with tf.variable_scope("ResNet") as scope:
  pred = ResNet(train_data, True)
  scope.reuse_variables()
  valpred = ResNet(val_data, False)

with tf.name_scope('Loss_and_Accuracy'):
  update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
  with tf.control_dependencies(update_ops):
    cost = tf.losses.sparse_softmax_cross_entropy(labels=train_labels, logits=pred)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    
  correct_prediction = tf.equal(tf.argmax(pred, 1), train_labels)
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  top5=tf.reduce_mean(tf.cast(tf.nn.in_top_k(pred, train_labels, 5), tf.float32))
  
  correct_prediction2 = tf.equal(tf.argmax(valpred, 1), val_labels)
  accuracy2 = tf.reduce_mean(tf.cast(correct_prediction2, tf.float32))
  
  tf.summary.scalar('Loss', cost)
  tf.summary.scalar('Training_Accuracy', accuracy)
  tf.summary.scalar('Top-5_accuracy', top5)

Preparing the training & validation data...


W0730 10:45:51.262284 140396069877632 deprecation.py:323] From <ipython-input-5-fbefbd28a462>:53: slice_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(tuple(tensor_list)).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0730 10:45:51.269682 140396069877632 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/input.py:374: range_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0730 10:45:51.291357 140396069877632 deprecation.py:3

Preparing the training model with learning rate = 0.01000...


W0730 10:45:52.835245 140396069877632 deprecation.py:323] From <ipython-input-4-0a28748977c8>:11: batch_normalization (from tensorflow.python.layers.normalization) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).


[L-1] Build 0th residual block 0 with 96 channels
[L-2] Build 0th connection layer 1 from 96 to 128 channels
[L-3] Build 1th residual block 0 with 128 channels
[L-4] Build 1th connection layer 1 from 128 to 256 channels
[L-5] Build 2th residual block 0 with 256 channels
[L-6] Build 2th connection layer 1 from 256 to 384 channels
[L-7] Build 3th residual block 0 with 384 channels
[L-8] Build 3th residual block 1 with 384 channels
[L-1] Build 0th residual block 0 with 96 channels
[L-2] Build 0th connection layer 1 from 96 to 128 channels
[L-3] Build 1th residual block 0 with 128 channels
[L-4] Build 1th connection layer 1 from 128 to 256 channels

W0730 10:45:54.379867 140396069877632 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/losses/losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



[L-5] Build 2th residual block 0 with 256 channels
[L-6] Build 2th connection layer 1 from 256 to 384 channels
[L-7] Build 3th residual block 0 with 384 channels
[L-8] Build 3th residual block 1 with 384 channels


In [0]:
saver = tf.train.Saver()
init = tf.global_variables_initializer()
sess.run(init)
step = 0
writer = tf.summary.FileWriter("log2", sess.graph)
summaries = tf.summary.merge_all()

print("We are going to train the ImageNet model based on ResNet!!!")
while (step * batch_size) < max_iter:
    epoch1=np.floor((step*batch_size)/glen1)
    if (((step*batch_size)%glen1 < batch_size) & (lr==1e-3) & (epoch1 >2)):
        lr /= 10

    sess.run(optimizer,  feed_dict={learning_rate: lr})

    if (step % 15000==1) & (step>15000):
        save_path = saver.save(sess, "./tf_resnet_model_iter" + str(step) + ".ckpt")
        print("Model saved in file at iteration %d: %s" % (step*batch_size,save_path))

    if step>1 and step % display_step == 1:
        # calculate the loss
        loss, acc, top5acc, summaries_string = sess.run([cost, accuracy,top5, summaries])
        print("Iter=%d/epoch=%d, Loss=%.6f, Training Accuracy=%.6f, Top-5 Accuracy=%.6f, lr=%f" % (step*batch_size, epoch1 ,loss, acc, top5acc, lr))
        writer.add_summary(summaries_string, step)
        
    if step>1 and (step % (display_step*10) == 1):
        rounds = tlen1 // batch_size
        valacc=[]
        for k in range(rounds):
          a2 = sess.run(accuracy2)
          valacc.append(a2)
        print("\nIter=%d/epoch=%d, Validation Accuracy=%.6f" % (step*batch_size, epoch1 , np.mean(valacc)))

  
    step += 1
print("Optimization Finished!")
save_path = saver.save(sess, "./tf_resnet_model.ckpt")
print("Model saved in file: %s" % save_path)

We are going to train the ImageNet model based on ResNet!!!
Iter=20736/epoch=0, Loss=3.175990, Training Accuracy=0.152344, Top-5 Accuracy=0.480469, lr=0.010000
Iter=41216/epoch=0, Loss=3.028897, Training Accuracy=0.238281, Top-5 Accuracy=0.503906, lr=0.010000
Iter=61696/epoch=0, Loss=2.789783, Training Accuracy=0.261719, Top-5 Accuracy=0.585938, lr=0.010000
Iter=82176/epoch=1, Loss=2.572429, Training Accuracy=0.289062, Top-5 Accuracy=0.636719, lr=0.010000
Iter=102656/epoch=1, Loss=2.382948, Training Accuracy=0.332031, Top-5 Accuracy=0.695312, lr=0.010000
Iter=123136/epoch=1, Loss=2.321332, Training Accuracy=0.343750, Top-5 Accuracy=0.699219, lr=0.010000
Iter=143616/epoch=2, Loss=2.136219, Training Accuracy=0.375000, Top-5 Accuracy=0.753906, lr=0.010000
Iter=164096/epoch=2, Loss=1.931998, Training Accuracy=0.414062, Top-5 Accuracy=0.792969, lr=0.010000
Iter=184576/epoch=2, Loss=1.937448, Training Accuracy=0.429688, Top-5 Accuracy=0.785156, lr=0.010000
Iter=205056/epoch=3, Loss=1.791667,

In [0]:
exit()